In [2]:
from __future__ import print_function
import warnings
from os import environ
from PIL import Image

Image.MAX_IMAGE_PIXELS = None
warnings.simplefilter('ignore', Image.DecompressionBombWarning)
environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras.layers import Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.layers import AveragePooling2D, Input, Flatten, Activation, Dropout, Dense
from keras.optimizers import Adam
from keras.initializers import glorot_normal
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.regularizers import l2
from keras import backend as K
from keras.models import Sequential
import pandas as pd
import random
import numpy as np
import shutil

In [3]:
def sve_jpg(df):
    for item in list(df.new_filename):
        if not '.jpg' in item:
            return False
    return True

In [4]:
df = pd.read_csv('/home/ivana/repos/Vje-ba/all_data_info.csv')
seed = 123

print(df.shape)
df.head()

(103250, 12)


,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg


In [5]:
print('Sve su jpg: ' + str(sve_jpg(df)))

Sve su jpg: True


In [6]:
threshold = 300

x = list(df['artist'].value_counts())
# broj umjetnika koji imaju vise ili jednako od 300 slika
print(len([a for a in x if a >= threshold]))
# len(set(x)) #---> ukupan broj umjetnika

57


In [7]:
# train, validation, test --- 80, 10, 10
num_train = 240
num_val = 30
num_test = num_val
num_samples = num_train + num_val + num_test
b_size = 60

#lista umjetnika koje ćemo promatrati
temp = df['artist'].value_counts()
artists = temp[temp >= threshold].index.tolist()
# print(artists)

num_artists = len(artists)
print('Prepoznajemo ' + str(num_artists) + ' umjetnika')

Prepoznajemo 57 umjetnika


In [7]:
train_dfs = []
val_dfs = []
test_dfs = []

for a in artists:
    # PROVJERI KASNIJE ŠTA JE S NA=TRUE
    tmp = df[df['artist'].str.startswith(a)].sample(n=num_samples, random_state=seed)
    # print(tmp.shape)
    t_df = tmp.sample(n=num_train, random_state=seed)
    rest_df = tmp.loc[~tmp.index.isin(t_df.index)] # uzmi komplement od t_df
    # print(rest_df.shape)
    v_df = rest_df.sample(n=num_val, random_state=seed)
    te_df = rest_df.loc[~rest_df.index.isin(v_df.index)]
    
    train_dfs.append(t_df)
    val_dfs.append(v_df)
    test_dfs.append(te_df)
    
    # ovo se pokrene samo jednom!!
    copyImagesToFiles(a, t_df, v_df, te_df)

train_df = pd.concat(train_dfs)
val_df = pd.concat(val_dfs)
test_df = pd.concat(test_dfs)

print('train tablica\t\t', train_df.shape)
print('validation tablica\t', val_df.shape)
print('test tablica\t\t', test_df.shape)

Prepoznajemo 57 umjetnika
train tablica		 (13680, 12)
validation tablica	 (1710, 12)
test tablica		 (1710, 12)


In [8]:
def center_crop(img, center_crop_size):
    assert img.shape[2] == 3
    centerw, centerh = img.shape[0] // 2, img.shape[1] // 2
    halfw, halfh = center_crop_size[0] // 2, center_crop_size[1] // 2
    return img[centerw-halfw:centerw+halfw, centerh-halfh:centerh+halfh, :]

# https://jkjung-avt.github.io/keras-image-cropping/
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length, random_cropping=True):
    '''
    Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator
    '''
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            if random_cropping == True:
                batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
            else:
                batch_crops[i] = center_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)


In [10]:
# velicina slika koje dajemo ulaznom sloju mreze
input_shape = (224, 224, 3)
# velicina batch-a
b_size = 30

train_datagen = ImageDataGenerator(
                horizontal_flip=True)

val_datagen = ImageDataGenerator(
                horizontal_flip=True)
test_datagen = ImageDataGenerator()


train_generator = train_datagen.flow_from_directory(
                    '../train',
                    batch_size=b_size,
                    class_mode='categorical')
train_generator = train_datagen.standardize(train_generator)
# na slikama iz train skupa radimo crop na slučajnom mjestu
train_crops = crop_generator(train_generator, 224)

validation_generator = val_datagen.flow_from_directory(
                    '../validation',
                    batch_size=b_size,
                    class_mode='categorical')
# na slikama iz validation skupa radimo centralni crop
val_crops = crop_generator(validation_generator, 224, False)


test_generator = test_datagen.flow_from_directory(
                '../test',
                target_size=(224, 224),
                batch_size=b_size)
test_crops = crop_generator(test_generator, 224, False)

Found 13680 images belonging to 57 classes.
Found 1710 images belonging to 57 classes.
Found 1710 images belonging to 57 classes.


Model mreže

In [14]:
# model mreže inspiriran glavnim člankom

model = Sequential()

model.add(Conv2D(32, 
                 kernel_size=3, 
                 strides=2, 
                 padding='same', 
                 input_shape=input_shape,
                 kernel_initializer=glorot_normal()))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32, 
                 kernel_size=3, 
                 strides=2, 
                 padding="same", 
                 input_shape=input_shape,
                 kernel_initializer=glorot_normal()))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(4*num_artists, 
                input_shape=(6272,),
                kernel_initializer=glorot_normal()))
model.add(Activation('relu'))
model.add(Dense(num_artists, 
                input_shape=(4*num_artists,),
                activation='softmax',
                kernel_initializer=glorot_normal()))

tbCallBack = TensorBoard(log_dir='./Graph300-5', 
                         histogram_freq=0, 
                         write_graph=True, 
                         write_images=True)

# koristimo adamov optimizator i metrika je točnost
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4),
              metrics=['accuracy'])

# crta tablicu slojeva mreže
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 112, 112, 32)      896       
_________________________________________________________________
activation_7 (Activation)    (None, 112, 112, 32)      0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 28, 28, 32)        128       
__________

In [16]:
# treniramo mrežu....

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

model.fit_generator(train_crops,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=10,
                    validation_data=val_crops,
                    validation_steps=STEP_SIZE_VALID,
                    workers=4,
                    callbacks=[tbCallBack])

Epoch 1/10
  5/456 [..............................] - ETA: 27:54 - loss: 5.2070 - acc: 0.0400

KeyboardInterrupt: 

In [17]:
model.save_weights('prvi_pokusaj300SVE-SLIKE20.h5')

In [18]:
evaluation = model.evaluate_generator(test_generator,
                         steps=test_generator.n//test_generator.batch_size,
                         workers=4,
                         verbose=1)

print(model.metrics_names)
print(evaluation)

57/57 [==============================] - 97s 2s/step
['loss', 'acc']
[8.563225168930856, 0.027485381354365433]


In [19]:
predictions = model.predict_generator(test_generator,
                        steps=test_generator.n//test_generator.batch_size,
                        workers=4,
                        verbose=1)

57/57 [==============================] - 83s 1s/step


In [20]:
preds = np.argmax(predictions, axis=1)

print(preds == test_generator.classes)
LIST = preds == test_generator.classes
print(sum(LIST)*100/len(LIST))
print(preds)

[False False False ... False False False]
1.4619883040935673
[43 27 19 ... 19 19 12]
